In [1]:
from functools import cached_property
import re

TODAY = 'day14'
TEST_FILE_INPUT = f"./test_input_{TODAY}.txt"
FILE_INPUT = f"./input_{TODAY}.txt"


In [161]:
MOVE_DIRECTION_DICT = {
    "north": (-1,0),
    "south": (1,0),
    "east": (0,1),
    "west": (0,-1)
}

class Board:
    def __init__(self, file_path):
        self.file_path = file_path
        
        
    @cached_property
    def n_rows(self):
        with open(self.file_path, 'r') as f:
            return len(f.readlines())
        
    @cached_property
    def n_cols(self):
        with open(self.file_path, 'r') as f:
            line = f.readline()
            line = line.rstrip('\n')
            return len(line)
        
    def find_positions(self, search_char):
        char_positions = []
        
        with open(self.file_path, 'r') as f:
            for ii, line in enumerate(f):
                line = line.rstrip('\n')
                for jj, char in enumerate(line):
                    if char == search_char:
                        char_positions.append((ii, jj))
                        
        return char_positions
        
    @cached_property
    def round_positions(self):
        return self.find_positions('O')
    
    @cached_property
    def cube_positions(self):
        positions = self.find_positions('#')
        for jj in range(self.n_cols):
            positions.append((-1,jj))
            positions.append((self.n_rows,jj))
            
        for ii in range(self.n_rows):
            positions.append((ii, -1))
            positions.append((ii, self.n_cols))
        return positions
                        
    def tilt(self, direction):
        move_direction = MOVE_DIRECTION_DICT[direction]
        round_positions_start = (x for x in self.round_positions)
        round_positions_loc = tuple([x for x in self.round_positions])
        # print(round_positions_start)
        for ball_loc in round_positions_start:
            
            # print(f"Rolling ball at {ball_loc} {direction}")
            x_curr, y_curr = ball_loc
            round_positions_loc = tuple([x for x in round_positions_loc if x != ball_loc])
            last_open_x, last_open_y = ball_loc
            dx, dy = move_direction
            found_cube = False
            while found_cube == False:
                x_curr += dx
                y_curr += dy
                if (x_curr, y_curr) in self.cube_positions:
                    found_cube = True
                    # print(f"  Stopped at cube {x_curr, y_curr}")
                if (x_curr, y_curr) not in list(self.round_positions) + self.cube_positions:
                    last_open_x, last_open_y = x_curr, y_curr
                        
            # print(f"  Placed ball at {last_open_x, last_open_y}")
                        
            round_positions_loc = tuple(list(round_positions_loc) + [(last_open_x, last_open_y)])
            # print(round_positions_loc)
            
            self.round_positions = round_positions_loc
            # self.display()
        
    def display(self):
        for ii in range(self.n_rows):
            row_str = ''
            for jj in range(self.n_cols):
                if (ii, jj) in self.round_positions:
                    row_str += 'O'
                elif (ii, jj) in self.cube_positions:
                    row_str += '#'
                else:
                    row_str += '.'
                    
            print(row_str)
            
    def north_load(self):
        value = 0
        for ii, jj in self.round_positions:
            value += self.n_rows - ii
        return value
                    
                
                

                

        

class PartA:
    def __init__(self, board):
        self.board = board
        

    def solve(self):
        self.board.tilt('north')
        return self.board.north_load()


In [162]:
board_test = Board(TEST_FILE_INPUT)

In [163]:
a_test = PartA(board_test)

In [164]:
assert a_test.solve() == 136 # NUMER HERE

In [165]:
board = Board(FILE_INPUT)

In [166]:
a = PartA(board)
a.solve()

110128

In [317]:
class PartB(PartA):
    def solve(self):
        positions = list(self.board.round_positions)
        positions.sort()
        seen_round_positions = {tuple(positions): 0}
        boards_dict = {0: self.board.north_load()}
        num_cycles = 1000000000
        skip = False
        cycle = 0
        while cycle < num_cycles and skip == False: 
            # print(ii, self.board.north_load())
            cycle += 1
            for direction in ('north', 'west', 'south', 'east'):
                self.board.tilt(direction)
            boards_dict[cycle] = self.board.north_load()
            new_round = list(self.board.round_positions)
            new_round.sort()
            new_round = tuple(new_round)
            
            if seen_round_positions.get(new_round, 0) != 0:
                skip = True
                print(f"Entered cycle after {cycle} rounds")
                print(f"Round positions for cycle {cycle} are the same as the ones for cycle {seen_round_positions.get(new_round)}")
            
            else:
                seen_round_positions[new_round] = cycle
                
                
                    
            # self.board.display()
            # print('\n')
        repeat_cycle_start = seen_round_positions.get(new_round)
        repeat_cycle_end = cycle - 1
        repeat_cycle_length = repeat_cycle_end - repeat_cycle_start + 1

        board_index_at_end = (num_cycles - repeat_cycle_end - 1) % repeat_cycle_length
        board_index_at_end += repeat_cycle_start

        print(f"So we end at the {board_index_at_end}th board")
        finishing_board = boards_dict[board_index_at_end]
        
        print({key: boards_dict[key] for key in boards_dict.keys()})

            
            
        return finishing_board#.north_load()

            
            

In [321]:
board_test = Board(TEST_FILE_INPUT)
b_test = PartB(board_test)


In [322]:
assert b_test.solve() == 64 # NUMBER HERE

Entered cycle after 10 rounds
Round positions for cycle 10 are the same as the ones for cycle 3
So we end at the 6th board
{0: 104, 1: 87, 2: 69, 3: 69, 4: 69, 5: 65, 6: 64, 7: 65, 8: 63, 9: 68, 10: 69}


In [324]:
board = Board(FILE_INPUT)
b = PartB(board)
b.solve()

Entered cycle after 191 rounds
Round positions for cycle 191 are the same as the ones for cycle 177
So we end at the 188th board
{0: 102449, 1: 101575, 2: 101554, 3: 101559, 4: 101711, 5: 101898, 6: 101991, 7: 102153, 8: 102343, 9: 102446, 10: 102549, 11: 102634, 12: 102711, 13: 102756, 14: 102802, 15: 102855, 16: 102931, 17: 103011, 18: 103139, 19: 103265, 20: 103397, 21: 103527, 22: 103627, 23: 103769, 24: 103894, 25: 104050, 26: 104239, 27: 104396, 28: 104537, 29: 104694, 30: 104829, 31: 104968, 32: 105118, 33: 105280, 34: 105431, 35: 105587, 36: 105693, 37: 105816, 38: 105922, 39: 106016, 40: 106089, 41: 106175, 42: 106282, 43: 106376, 44: 106421, 45: 106495, 46: 106566, 47: 106625, 48: 106676, 49: 106759, 50: 106863, 51: 106962, 52: 107042, 53: 107094, 54: 107110, 55: 107124, 56: 107169, 57: 107218, 58: 107290, 59: 107347, 60: 107368, 61: 107395, 62: 107392, 63: 107398, 64: 107410, 65: 107432, 66: 107466, 67: 107485, 68: 107482, 69: 107484, 70: 107481, 71: 107486, 72: 107491, 73: 

103861